# ABR Counterfeit Detection System

# Rs.1000

In [1]:
# Importing all necessary libraries
import cv2                            # Importing Opencv library for image processing and computer vision
import numpy as np                    # Importing numpy library 
import matplotlib.pyplot as plt       # Importing matplotlib library to plot the images directly in notebook
from skimage.metrics import structural_similarity as ssim   # Importing ssim calculating modules from skimage library

# Importing tkinter library to build GUI
from tkinter import *
from tkinter.ttk import Progressbar

import time

#Resizing the Plots
plt.rcParams["figure.figsize"] = (12, 12)

In [2]:
# Declaring variable for progress bar to store the total progress
myProgress = 0.0

In [ ]:
# This ipython magic will retrieve the path of the input image which is stored when the input_gui.ipynb is executed.

# %store -r path

# A sample path:
# path = 'user/path.jpg'

print('Path of input image: ', path)

In [4]:
# Reading the image
test_img = cv2.imread(path)

# Pre-processing

In [5]:
# Pre-processing

# Resizing the image
test_img = cv2.resize(test_img, (1165,455))

# Guassian Blur
blur_test_img = cv2.GaussianBlur(test_img, (5,5), 0)

# Grayscale conversion
gray_test_image = cv2.cvtColor(blur_test_img, cv2.COLOR_BGR2GRAY)

def preprocessing():
    # Showing original currency note
    plt.imshow(gray_test_image, 'gray')
    plt.title('Input image after pre-processing')
    plt.show()
    
    # Updating the progress
    progress['value']=5
    ProgressWin.update_idletasks()

In [6]:
#  Calculating SSIM of the two images sent as parameters
def calculateSSIM(template_img, query_img):
    
    min_w = min(template_img.shape[1], query_img.shape[1])
    min_h = min(template_img.shape[0], query_img.shape[0])
    
    # Resizing the two images so that both have same dimensions
    img1 = cv2.resize(template_img, (min_w, min_h))
    img2 = cv2.resize(query_img, (min_w, min_h))
    
    # Conversion to gray-scale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    # Plotting the images
    plt.subplot(1, 2, 1)
    plt.imshow(img1, 'gray')
    
    plt.subplot(1, 2, 2)
    plt.imshow(img2, 'gray')
    
    plt.show()
    
    # Find the SSIM score and return
    score = ssim(img1, img2)
    return score

In [7]:
# Feature detection using ORB
def computeORB(template_img, query_img):
    # Creating orb object 
    nfeatures=700;
    scaleFactor=1.2;
    nlevels=8;
    edgeThreshold=15; # Changed default (31);

    # Initialize the ORB detector algorithm 
    orb = cv2.ORB_create(
        nfeatures,
        scaleFactor,
        nlevels,
        edgeThreshold)
    
    # Find the keypoints and descriptors with ORB
    # This will find the keypoints of each of the image and then find the descriptors corresponding to each keypoint.
    
    kpts1, descs1 = orb.detectAndCompute(template_img,None)
    kpts2, descs2 = orb.detectAndCompute(query_img,None)
    
    # Brute Force Matching 
    # Starting a brute force matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Finding matches between the 2 descrptor sets
    matches = bf.match(descs1, descs2)

    # sort the matches in the order of their distance
    # Lower the distance, better the matching
    dmatches = sorted(matches, key = lambda x:x.distance)
    
    # Image homography
    ## extract the matched keypoints
    src_pts  = np.float32([kpts1[m.queryIdx].pt for m in dmatches]).reshape(-1,1,2)
    dst_pts  = np.float32([kpts2[m.trainIdx].pt for m in dmatches]).reshape(-1,1,2)
        
    ## find homography matrix and do perspective transform
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    h,w = template_img.shape[:2]
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    
    if M is not None:
        dst = cv2.perspectiveTransform(pts,M)
    else:
        dst = None

    # This finds the template region in the test currency note
    
    # Returning necessary data
    return dst, dst_pts, kpts1, kpts2, dmatches

# Features 1- 7: Using ORB and SSIM

In [8]:
# Values for specifying search area of features 1 to 7
search_area_list = [[200,270,160,330],
                    [1050,1500,250,400],
                    [50,400,0,100],
                    [750,1050,0,100],
                    [850,1050,280,380],
                    [700,820,290,370],
                    [400,650,0,100]]

# Values of max_area and min_area for each feature for features 1 to 7
feature_area_limits_list = [[10000,14000],
                            [9000,15000],
                            [17000,21500],
                            [19000,28000],
                            [17500,23000],
                            [6500,9000],
                            [10000,16000]]

score_set_list = []               # Stores the ssim score set of each feature
best_extracted_img_list = []      # Stores the extracted image with highest SSIM score for each feature
avg_ssim_list = []                # Stores the avg ssim value for each feature
NUM_OF_FEATURES = 7               # Number of features

In [9]:
# Verification of features 1 to 7
def testFeature_1_2_7():
    i = 0
    j = 0
    NUMBER_OF_TEMPLATES = 6
    global score_set_list                # Stores the ssim score set of each feature
    global best_extracted_img_list       # Stores the extracted image with highest SSIM score for each feature
    global avg_ssim_list                 # Stores the avg ssim value for each feature
    
    #Progress bar
    global myProgress
    myProgress =progress['value']
    
    # Iterating for each feature
    for j in range(NUM_OF_FEATURES):
        print('ANALYSIS OF FEATURE ' + str(j+1))

        score_set = []           # SSIM scores for each teamplate of current feature will be stored here
        max_score = -1           # Stores max SSIM score
        max_score_img = None     # Stores extraced image with max SSIM score for the current feature
        
        # Performing feature detection, extraction and comparison for each template stored in dataset 
        for i in range(NUMBER_OF_TEMPLATES):
            print('Template ' + str(i+1) + ' :')
            
            # Current template 
            template_path = r'Feature_Dataset/Feature ' + str(j+1) + '//' + str(i+1) + '.jpg'
            template_img = cv2.imread(template_path)

            template_img_blur = cv2.GaussianBlur(template_img, (5,5), 0)
            template_img_gray = cv2.cvtColor(template_img_blur, cv2.COLOR_BGR2GRAY)
            test_img_mask = gray_test_image.copy()
            
            # Creating a mask to search the current template.
            search_area = search_area_list[j]

            test_img_mask[:, :search_area[0]] = 0
            test_img_mask[:, search_area[1]:] = 0
            test_img_mask[:search_area[2], :] = 0
            test_img_mask[search_area[3]:, :] = 0
            
            # Feature detection using ORB 
            dst, dst_pts, kpts1, kpts2, dmatches = computeORB(template_img_gray, test_img_mask)
            
            # Error handling
            if dst is None:
                print('An Error occurred - Homography matrix is of NoneType')
                continue
            
            query_img = test_img.copy()
            
            # drawing polygon around the region where the current template has been detected on the test currency note -- the blue polygon
            res_img1 = cv2.polylines(query_img, [np.int32(dst)], True, (0,0,255), 1, cv2.LINE_AA)

            # draw match lines between the matched descriptors
            res_img2 = cv2.drawMatches(template_img, kpts1, res_img1, kpts2, dmatches[:20],None,flags=2)

            # Find the details of a bounding rectangle that bounds the above polygon --- green rectangle
            (x, y, w, h) = cv2.boundingRect(dst) # This gives us details about the rectangle that bounds this contour  
            
            # Checking if the area of the detected region is within the min and max area allowed for current feature 
            min_area = feature_area_limits_list[j][0]
            max_area = feature_area_limits_list[j][1]

            feature_area = w*h

            if feature_area < min_area or feature_area > max_area:
                (x, y, w, h) = cv2.boundingRect(dst_pts) 

                feature_area = w*h
                if feature_area < min_area or feature_area > max_area: 
                    # If even area of 2nd rect is outside limits, then Discard current template
                    print('Template Discarded- Area of extracted feature is outside permitted range!')
                    continue

            # Draw the rectangle
            cv2.rectangle(res_img1, (x,y), (x+w, y+h), (0,255,0), 3)
            
            # Plotting images
            plt.rcParams["figure.figsize"] = (16, 16)
            plt.subplot(1, 2, 1)
            plt.imshow(res_img2)

            plt.subplot(1, 2, 2)
            plt.imshow(res_img1)
            plt.show()
            
            # SSIM calculation
            # Crop out the region inside the green rectangle (matched region)
            crop_img = blur_test_img[y:y+h, x:x+w]

            plt.rcParams["figure.figsize"] = (5, 5)
            score = calculateSSIM(template_img_blur, crop_img)

            score_set.append(score)
            print('SSIM score: ', score, '\n')
            
            # Keeping details about extracted region with highest SSIM score
            if score > max_score:
                max_score = score
                max_score_img = crop_img
                
            #Progress bar- Updating the progess
            myProgress = myProgress + (75.0/(NUM_OF_FEATURES*NUMBER_OF_TEMPLATES))
            progress['value'] = myProgress 
            ProgressWin.update_idletasks()
            
        # Storing necessary data
        score_set_list.append(score_set)
        print('SSIM score set of Feature ' + str(j+1) + ': ', score_set, '\n')
        
        if len(score_set) != 0:
            avg_ssim_list.append(sum(score_set)/len(score_set))
            print('Average SSIM of Feature ' + str(j+1) + ': ',sum(score_set)/len(score_set),'\n')
        else:
            print('No SSIM scores were found for this feature!')
            avg_ssim_list.append(0.0)
            print('Average SSIM of Feature ' + str(j+1) + ': 0','\n')
        
        best_extracted_img_list.append([max_score_img, max_score])

    # Printing all details for features 1 - 7
    print('Final Score - set list:','\n')

    for x in range(len(score_set_list)):
        print('Feature', x+1,':',score_set_list[x])
    print('\n')

    print('Final Average SSIM list for each feature:','\n')

    for x in range(len(avg_ssim_list)):
        print('Feature', x+1,':',avg_ssim_list[x])


# Results

In [18]:
# Result analysis
result_list = []
def testResult():
    plt.rcParams["figure.figsize"] = (3, 3)
    
    print('\nRESULT ANALYSIS\n')
    
    # Stores the min allowed SSIM score for each feature
    min_ssim_score_list = [0.45, 0.4, 0.45, 0.45, 0.5, 0.4, 0.5]

    global result_list
    result_list = []               # To store details of each feature
    successful_features_count = 0  # To store number of features which passed the test

    # Feature 1 to 7: Results
    for i in range(NUM_OF_FEATURES):
        avg_score = avg_ssim_list[i]
        img, max_score = best_extracted_img_list[i]
        status = False
        min_allowed_score = min_ssim_score_list[i]
        
        # A feature passes the test if its avg SSIM score is greater than a min. decided value 
        # or if its max SSIM score is greater than 0.8
        if avg_score >= min_allowed_score or max_score >= 0.79:
            status = True
            successful_features_count += 1
            print('Feature ' + str(i+1) + ': Successful')
        else:
            status = False
            print('Feature ' + str(i+1) + ': Unsuccessful')
        
        if img is None:
            img = cv2.imread('error404.jpg')
        result_list.append([img, avg_score, max_score, status])

    print('\nResult Summary:')
    print(str(successful_features_count) + ' out of 7 features are VERIFIED!')
    
    # Updating progress bar
    global myProgress
    progress['value']=97
    ProgressWin.update_idletasks()

# Main cell

In [ ]:
# Main cell - contains GUI elements
# Call all testing functions
def Testing():  
    button.config(state = DISABLED)
    result_list.clear()
    preprocessing()
    testFeature_1_2_7()
    testResult()
    progress['value'] = 100
    ProgressWin.update_idletasks()
    time.sleep(0.8)
    ProgressWin.destroy()

def exitGUI():
    ProgressWin.destroy()


# creating tkinter window 
ProgressWin = Tk() 
ProgressWin.title("Processing Image")
ProgressWin.title('ABR Counterfeit Detection System - Processing')

# Defining attributes of root window
ProgressWin.resizable(False, False)  # This code helps to disable windows from resizing

window_height = 200
window_width = 500

screen_width = ProgressWin.winfo_screenwidth()
screen_height = ProgressWin.winfo_screenheight()

x_cordinate = int((screen_width/2) - (window_width/2))
y_cordinate = int((screen_height/2) - (window_height/2))

ProgressWin.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

# Creating a main frame inside the root window
main_frame=Frame(ProgressWin,relief=GROOVE)
main_frame.place(x=10,y=10) # Placing the frame at (10, 10)

# Creating sub-frames
frame1 = Frame(main_frame, padx=3, pady=3)
frame2 = Frame(main_frame, bg='dark blue', pady=5, padx = 5)
frame3 = Frame(main_frame, pady=5, padx = 5)

frame1.grid(row = 1, column = 1, padx = 5, pady = 5)
frame2.grid(row = 2, column = 1, padx = 5, pady = 5)
frame3.grid(row = 3, column = 1, padx = 5, pady = 30)

# Title label in sub_frame1
label = Label(master=frame1, text="Please wait... Processing is going on!", fg = 'green', font = "audiowide 13 bold")
label.pack() # Put the label into the window

# Progress bar widget 
progress = Progressbar(frame2, orient = HORIZONTAL, length = 450, mode = 'determinate') 
progress.pack()

# Button widget
button = Button(frame3, text = 'Click to continue', command = Testing, font = "audiowide 12 bold", pady = 5)
button.pack()

# Run the GUI  
ProgressWin.mainloop()

In [ ]:
# Store the list containing the final result of each feature
%store result_list